In [28]:
# Kimlik Doğrulama.
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


# Otokodlayıcılar ile Gürültü Giderme


In [29]:
# ilgili kütüphanelerin kurulması
from keras.layers import Input, Dense, Conv2D, MaxPooling2D,UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.datasets import mnist
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt 

In [30]:
# veriye kümesini oluşturmak ve ayırmak
(x_train, _), (x_test,_) = mnist.load_data()

In [31]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = np.reshape (x_train, (len(x_train), 28, 28, 1))
x_test =  np.reshape (x_test, (len(x_test), 28, 28, 1))

In [ ]:
print('Train: ',x_train.shape)
print('Test: ', x_test.shape)

In [33]:
# veriye gürültü ekleme
noise_factor = 0.5

x_train_noise = x_train + noise_factor * np.random.normal(loc = 0.0, scale = 1.0, size= x_train.shape)
x_test_noise = x_test + noise_factor * np.random.normal(loc = 0.0, scale = 1.0, size= x_test.shape)

In [34]:
x_train_noise = np.clip(x_train_noise, 0., 1.)
x_test_noise = np.clip(x_test_noise, 0., 1.)

In [ ]:
# gürültülü veriyi görselleştirme
%matplotlib inline
n = 10
plt.figure(figsize=(25,4))
for i in range(n):
  ax = plt.subplot(2,n, i+1)
  plt.imshow(x_test_noise[i].reshape(28,28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()
print('gürültülü orjinal görüntüler.')

In [36]:
# encode modeli.
''' bu kısımda genelde ya convolution ya da pooling yapılır.'''
input_img = Input(shape = (28,28,1))

x = Conv2D(32, (3,3), activation='relu', padding='same')(input_img)

x = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)

encoded = MaxPooling2D((2,2),padding='same')(x)

# daha önceki (cnn) bölümde hesaplamayı görmüştük.
# hesaplamaya göre 7*7*32 lik yeni tensörü elde etmemiz gerekecektir.


In [37]:
# decode modeli.
''' burada en başta 28*28*1 lik görüntüyü 7*7*32 ye çevirmiştik. şimdi tekrardan 28*28*1 e çevirmek istiyoruz.'''
x = Conv2D(32, (3,3), activation='relu', padding='same')(encoded)

x = UpSampling2D((2,2))(x)

x = Conv2D(32, (3,3), activation='relu', padding='same')(x)

x = UpSampling2D((2,2))(x)

In [38]:
decoded = Conv2D(1, (3,3), activation = 'sigmoid', padding='same')(x)

In [39]:
# modelin derlenmesi.
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
from matplotlib.rcsetup import validate_animation_writer_path
autoencoder.fit(x_train_noise, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noise, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/tb', histogram_freq=0, write_graph=False)])

decoded_imgs = autoencoder.predict(x_test)

In [ ]:
%matplotlib inline
n = 10
plt.figure(figsize=(25,4))
# yeniden yapılandırmayı ekranda göster.
for i in range(n):
  ax = plt.subplot(2,n, i+1)
  plt.imshow(decoded_imgs[i].reshape(28,28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()
print('gürültüsü baştırılmış görüntü.')